In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: Train and deploy an XGBoost model with pre-built containers (formerly hosted runtimes)


## Installation

Install the latest (preview) version of Vertex SDK.


In [ ]:
! pip3 install -U google-cloud-aiplatform --user

Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex AI services](https://cloud.google.com/vertex-ai/docs/general/locations)


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.


In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.

- Model Service for managed models.
- Endpoint Service for deployment.
- Job Service for batch jobs and custom training.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.


In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

## Prepare a trainer script

### Package assembly

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
setuptools.setup(\n\
    install_requires=[\n\
    ],\n\
    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Custom XGBoost Iris\n\
Version: 0.0.0\n\
Summary: Demonstration training script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@google.com\n\
License: Public\n\
Description: Demo\n\
Platform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Task.py contents

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Iris

import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()

# Download data
iris_data_filename = 'iris_data.csv'
iris_target_filename = 'iris_target.csv'
data_dir = 'gs://cloud-samples-data/ai-platform/iris'

# gsutil outputs everything to stderr so we need to divert it to stdout.
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_data_filename),
                       iris_data_filename], stderr=sys.stdout)
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_target_filename),
                       iris_target_filename], stderr=sys.stdout)


# Load data into pandas, then use `.values` to get NumPy arrays
iris_data = pd.read_csv(iris_data_filename).values
iris_target = pd.read_csv(iris_target_filename).values

# Convert one-column 2D array into 1D array for use with XGBoost
iris_target = iris_target.reshape((iris_target.size,))


# Load data into DMatrix object
dtrain = xgb.DMatrix(iris_data, label=iris_target)

# Train XGBoost model
bst = xgb.train({}, dtrain, 20)

# Export the classifier to a file
model_filename = 'model.bst'
bst.save_model(model_filename)


# Upload the saved model file to Cloud Storage
gcs_model_path = os.path.join(args.model_dir, model_filename)
subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path],
    stderr=sys.stdout)


### Store training script on your Cloud Storage bucket

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/iris.tar.gz

## Train a model

### [projects.locations.customJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)

#### Request

In [ ]:
TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1:latest"

JOB_NAME = "custom_job_XGB" + TIMESTAMP

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": {"machine_type": "n1-standard-4"},
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": ["gs://" + BUCKET_NAME + "/iris.tar.gz"],
            "python_module": "trainer.task",
            "args": ["--model-dir=" + "gs://{}/{}".format(BUCKET_NAME, JOB_NAME)],
        },
    }
]

training_job = aip.CustomJob(
    display_name=JOB_NAME, job_spec={"worker_pool_specs": WORKER_POOL_SPEC}
)

print(
    MessageToJson(
        aip.CreateCustomJobRequest(parent=PARENT, custom_job=training_job).__dict__[
            "_pb"
        ]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "customJob": {
    "displayName": "custom_job_XGB20210323142337",
    "jobSpec": {
      "workerPoolSpecs": [
        {
          "machineSpec": {
            "machineType": "n1-standard-4"
          },
          "replicaCount": "1",
          "pythonPackageSpec": {
            "executorImageUri": "gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1:latest",
            "packageUris": [
              "gs://migration-ucaip-trainingaip-20210323142337/iris.tar.gz"
            ],
            "pythonModule": "trainer.task",
            "args": [
              "--model-dir=gs://migration-ucaip-trainingaip-20210323142337/custom_job_XGB20210323142337"
            ]
          }
        }
      ]
    }
  }
}
```


#### Call

In [ ]:
request = clients["job"].create_custom_job(parent=PARENT, custom_job=training_job)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/customJobs/7371064379959148544",
  "displayName": "custom_job_XGB20210323142337",
  "jobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4"
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1:latest",
          "packageUris": [
            "gs://migration-ucaip-trainingaip-20210323142337/iris.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://migration-ucaip-trainingaip-20210323142337/custom_job_XGB20210323142337"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-23T14:23:45.067026Z",
  "updateTime": "2021-03-23T14:23:45.067026Z"
}
```


In [ ]:
# The full unique ID for the custom training job
custom_training_id = request.name
# The short numeric ID for the custom training job
custom_training_short_id = custom_training_id.split("/")[-1]

print(custom_training_id)

### [projects.locations.customJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)

#### Call

In [ ]:
request = clients["job"].get_custom_job(name=custom_training_id)

#### Response

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/customJobs/7371064379959148544",
  "displayName": "custom_job_XGB20210323142337",
  "jobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4"
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1:latest",
          "packageUris": [
            "gs://migration-ucaip-trainingaip-20210323142337/iris.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://migration-ucaip-trainingaip-20210323142337/custom_job_XGB20210323142337"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-23T14:23:45.067026Z",
  "updateTime": "2021-03-23T14:23:45.067026Z"
}
```


In [ ]:
while True:
    response = clients["job"].get_custom_job(name=custom_training_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

# model artifact output directory on Google Cloud Storage
model_artifact_dir = (
    response.job_spec.worker_pool_specs[0].python_package_spec.args[0].split("=")[-1]
)
print("artifact location  " + model_artifact_dir)

## Deploy the model

### [projects.locations.models.upload](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models/upload)

#### Request

In [ ]:
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest"

model = {
    "display_name": "custom_job_XGB" + TIMESTAMP,
    "artifact_uri": model_artifact_dir,
    "container_spec": {"image_uri": DEPLOY_IMAGE, "ports": [{"container_port": 8080}]},
}

print(MessageToJson(aip.UploadModelRequest(parent=PARENT, model=model).__dict__["_pb"]))

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "custom_job_XGB20210323142337",
    "containerSpec": {
      "imageUri": "gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest",
      "ports": [
        {
          "containerPort": 8080
        }
      ]
    },
    "artifactUri": "gs://migration-ucaip-trainingaip-20210323142337/custom_job_XGB20210323142337"
  }
}
```


#### Call

In [ ]:
request = clients["model"].upload_model(parent=PARENT, model=model)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "model": "projects/116273516712/locations/us-central1/models/2093698837704081408"
}
```


In [ ]:
# The full unique ID for the model version
model_id = result.model

## Make batch predictions


### Make a batch prediction file


In [ ]:
import json

import tensorflow as tf

INSTANCES = [[1.4, 1.3, 5.1, 2.8], [1.5, 1.2, 4.7, 2.4]]

gcs_input_uri = "gs://" + BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for i in INSTANCES:
        f.write(str(i) + "\n")

! gsutil cat $gcs_input_uri

*Example output*:
```
[1.4, 1.3, 5.1, 2.8]
[1.5, 1.2, 4.7, 2.4]
```


### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)


#### Request


In [ ]:
model_parameters = Value(
    struct_value=Struct(
        fields={
            "confidence_threshold": Value(number_value=0.5),
            "max_predictions": Value(number_value=10000.0),
        }
    )
)

batch_prediction_job = {
    "display_name": "custom_job_XGB" + TIMESTAMP,
    "model": model_id,
    "input_config": {
        "instances_format": "jsonl",
        "gcs_source": {"uris": [gcs_input_uri]},
    },
    "model_parameters": model_parameters,
    "output_config": {
        "predictions_format": "jsonl",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
        },
    },
    "dedicated_resources": {
        "machine_spec": {"machine_type": "n1-standard-2"},
        "starting_replica_count": 1,
        "max_replica_count": 1,
    },
}

print(
    MessageToJson(
        aip.CreateBatchPredictionJobRequest(
            parent=PARENT, batch_prediction_job=batch_prediction_job
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "custom_job_XGB20210323142337",
    "model": "projects/116273516712/locations/us-central1/models/2093698837704081408",
    "inputConfig": {
      "instancesFormat": "jsonl",
      "gcsSource": {
        "uris": [
          "gs://migration-ucaip-trainingaip-20210323142337/test.jsonl"
        ]
      }
    },
    "modelParameters": {
      "max_predictions": 10000.0,
      "confidence_threshold": 0.5
    },
    "outputConfig": {
      "predictionsFormat": "jsonl",
      "gcsDestination": {
        "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210323142337/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}
```


#### Call


In [ ]:
request = clients["job"].create_batch_prediction_job(
    parent=PARENT, batch_prediction_job=batch_prediction_job
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/1415053872761667584",
  "displayName": "custom_job_XGB20210323142337",
  "model": "projects/116273516712/locations/us-central1/models/2093698837704081408",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210323142337/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "confidence_threshold": 0.5,
    "max_predictions": 10000.0
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210323142337/batch_output/"
    }
  },
  "dedicatedResources": {
    "machineSpec": {
      "machineType": "n1-standard-2"
    },
    "startingReplicaCount": 1,
    "maxReplicaCount": 1
  },
  "manualBatchTuningParameters": {},
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-23T14:25:10.582704Z",
  "updateTime": "2021-03-23T14:25:10.582704Z"
}
```


In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### [projects.locations.batchPredictionJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)


#### Call


In [ ]:
request = clients["job"].get_batch_prediction_job(name=batch_job_id)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/1415053872761667584",
  "displayName": "custom_job_XGB20210323142337",
  "model": "projects/116273516712/locations/us-central1/models/2093698837704081408",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210323142337/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "max_predictions": 10000.0,
    "confidence_threshold": 0.5
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210323142337/batch_output/"
    }
  },
  "dedicatedResources": {
    "machineSpec": {
      "machineType": "n1-standard-2"
    },
    "startingReplicaCount": 1,
    "maxReplicaCount": 1
  },
  "manualBatchTuningParameters": {},
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-23T14:25:10.582704Z",
  "updateTime": "2021-03-23T14:25:10.582704Z"
}
```


In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(
            response.output_config.gcs_destination.output_uri_prefix
        )
        ! gsutil ls $folder/prediction*

        ! gsutil cat -h $folder/prediction*
        break
    time.sleep(60)

*Example output*:
```
==> gs://migration-ucaip-trainingaip-20210323142337/batch_output/prediction-custom_job_XGB20210323142337-2021_03_23T07_25_10_544Z/prediction.errors_stats-00000-of-00001 <==

==> gs://migration-ucaip-trainingaip-20210323142337/batch_output/prediction-custom_job_XGB20210323142337-2021_03_23T07_25_10_544Z/prediction.results-00000-of-00001 <==
{"instance": [1.4, 1.3, 5.1, 2.8], "prediction": 2.0451931953430176}
{"instance": [1.5, 1.2, 4.7, 2.4], "prediction": 1.9618644714355469}
```


## Make online predictions

### [projects.locations.endpoints.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/create)


#### Request


In [ ]:
endpoint = {"display_name": "custom_job_XGB" + TIMESTAMP}

print(
    MessageToJson(
        aip.CreateEndpointRequest(parent=PARENT, endpoint=endpoint).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "endpoint": {
    "displayName": "custom_job_XGB20210323142337"
  }
}
```


#### Call


In [ ]:
request = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/endpoints/1733903448723685376"
}
```


In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### [projects.locations.endpoints.deployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/deployModel)


#### Request


In [ ]:
deployed_model = {
    "model": model_id,
    "display_name": "custom_job_XGB" + TIMESTAMP,
    "dedicated_resources": {
        "min_replica_count": 1,
        "max_replica_count": 1,
        "machine_spec": {"machine_type": "n1-standard-4", "accelerator_count": 0},
    },
}

print(
    MessageToJson(
        aip.DeployModelRequest(
            endpoint=endpoint_id,
            deployed_model=deployed_model,
            traffic_split={"0": 100},
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/1733903448723685376",
  "deployedModel": {
    "model": "projects/116273516712/locations/us-central1/models/2093698837704081408",
    "displayName": "custom_job_XGB20210323142337",
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-4"
      },
      "minReplicaCount": 1,
      "maxReplicaCount": 1
    }
  },
  "trafficSplit": {
    "0": 100
  }
}
```


#### Call


In [ ]:
request = clients["endpoint"].deploy_model(
    endpoint=endpoint_id, deployed_model=deployed_model, traffic_split={"0": 100}
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "deployedModel": {
    "id": "7407594554280378368"
  }
}
```


In [ ]:
# The unique ID for the deployed model
deployed_model_id = result.deployed_model.id

print(deployed_model_id)

### [projects.locations.endpoints.predict](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/predict)


### Prepare file for online prediction

In [ ]:
INSTANCES = [[1.4, 1.3, 5.1, 2.8], [1.5, 1.2, 4.7, 2.4]]

#### Request


In [ ]:
prediction_request = {"endpoint": endpoint_id, "instances": INSTANCES}

print(json.dumps(prediction_request, indent=2))

*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/1733903448723685376",
  "instances": [
    [
      1.4,
      1.3,
      5.1,
      2.8
    ],
    [
      1.5,
      1.2,
      4.7,
      2.4
    ]
  ]
}
```


#### Call


In [ ]:
request = clients["prediction"].predict(endpoint=endpoint_id, instances=INSTANCES)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "predictions": [
    2.045193195343018,
    1.961864471435547
  ],
  "deployedModelId": "7407594554280378368"
}
```


### [projects.locations.endpoints.undeployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/undeployModel)


#### Call


In [ ]:
request = clients["endpoint"].undeploy_model(
    endpoint=endpoint_id, deployed_model_id=deployed_model_id, traffic_split={}
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_model = True
delete_endpoint = True
delete_pipeline = True
delete_batchjob = True
delete_bucket = True

# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    if delete_model:
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex AI fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the custom training using the Vertex AI fully qualified identifier for the custome training
try:
    if custom_training_id:
        clients["job"].delete_custom_job(name=custom_training_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex AI fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME